In [54]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-29 10:13:08--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip.9’

dataset_tsmc2014.zi 100%[===================>]  24.36M  6.18MB/s    in 5.6s    

2024-05-29 10:13:14 (4.35 MB/s) - ‘dataset_tsmc2014.zip.9’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
replace dataset_tsmc2014/dataset_TSMC2014_NYC.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [55]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [56]:
NUMCHECKIN = 7000

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
36506        58  4a385663f964a520ed9e1fe3  4bf58dd8d48988d103951735   
96254       465  49d376a1f964a520fa5b1fe3  4bf58dd8d48988d1e0931735   
16267       293  4d2f5ae2940137044fa1eeda  4bf58dd8d48988d1f6931735   
88301       928  4afcad9ff964a520052522e3  4bf58dd8d48988d118951735   
149382      333  4c1a3215838020a109cfe661  4c38df4de52ce0d596b336e1   
...         ...                       ...                       ...   
70486       923  4f97a579e4b075734624c95b  4bf58dd8d48988d103941735   
177577       19  4a8de6b5f964a5205b1120e3  4bf58dd8d48988d1d2941735   
112256      913  4b0b68cef964a5202f3123e3  4bf58dd8d48988d1fd931735   
105967      484  4cca1c3982bfa35df6e250c9  4bf58dd8d48988d103941735   
9294        525  4f5684de771657f331e5ca01  4bf58dd8d48988d103941735   

       Venue category name   Latitude  Longitude  Timezone  \
36506       Clothing Store  40.726443 -73.995746      -240   
96254          Coffee S

In [57]:

checkin_counts = df['User ID'].value_counts() # Conta il numero di check-in per ogni utente
users_with_sufficient_checkins = checkin_counts[checkin_counts >= 5].index   # Filtra gli utenti con meno di 5 check-in
df = df[df['User ID'].isin(users_with_sufficient_checkins)]  # Mantieni solo le righe relative agli utenti con almeno 5 check-in

print(f"Numero di utenti con meno di 5 check-in: {len(checkin_counts[checkin_counts < 5])}")

Numero di utenti con meno di 5 check-in: 464


In [58]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
36506        58       585                  6  40.726443 -73.995746      -240   
96254       465       423                214  40.732876 -73.988193      -240   
16267       293      2692                247  40.689003 -73.929085      -240   
88301       928      1111                 43  40.795185 -73.931266      -240   
149382      333      2140                264  40.707209 -74.014259      -240   
...         ...       ...                ...        ...        ...       ...   
91008       753      1076                 70  40.736064 -74.029387      -240   
70486       923      3730                  5  40.652203 -74.129868      -240   
177577       19       797                202  40.752985 -74.026027      -300   
112256      913      1167                258  40.679336 -73.930285      -240   
105967      484      2552                  5  40.696558 -73.953616      -240   

                              UTC time 

In [59]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [60]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [61]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-61-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [62]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
numvenues = len(venuesdf)
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
36506        585  40.726443 -73.995746                  6
96254        423  40.732876 -73.988193                214
16267       2692  40.689003 -73.929085                247
88301       1111  40.795185 -73.931266                 43
149382      2140  40.707209 -74.014259                264
...          ...        ...        ...                ...
163389      1604  40.823166 -73.953159                149
62511       1296  40.757996 -73.989531                156
176500      3231  40.728996 -73.996746                175
217454       707  40.737025 -74.005487                121
177577       797  40.752985 -74.026027                202

[3949 rows x 4 columns]


In [63]:
!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html

Looking in indexes: https://download.pytorch.org/whl/
2.3.0+cu121
12.1
Looking in links: https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html


In [64]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append([float(el['Venue category ID'])])
  i+=1

print(len(node_attr))


3949


In [65]:
#import torch
sequences = {}
count = {}

MINVISIT = 5


for _, row in df.iterrows():

  if row['User ID'] not in sequences:
    sequences[row['User ID']] = [[row['Venue ID'],row['localHour'],row['localDayofWeek']]]


  else:
    sequences[row['User ID']].append([row['Venue ID'],row['localHour'],row['localDayofWeek']])


  if row['Venue ID'] not in count:
    count[row['Venue ID']] = 1
  else:
    count[row['Venue ID']] += 1

print(sequences)
seq = []
next = []
to_remove = []

for el in sequences:
  entry = [el]
  for i in range(len(sequences[el][len(sequences[el])-1])):
    el2 = sequences[el][len(sequences[el])-1][i]
    #print(el2,i)
    if i == 0:
      el2 = nodeIds[el2]
    #print(el2,i)
    entry.append(el2)
  for i in range(len(sequences[el])-1):
    entry.append(nodeIds[sequences[el][i][0]])
  seq.append(torch.tensor(entry))

  target = torch.zeros(numvenues)
  target[sequences[el][len(sequences[el])-1][0]] = 1
  next.append(target)
  #print(count[sequences[el][len(sequences[el])-1][0]])
  if count[sequences[el][len(sequences[el])-1][0]] < MINVISIT:

    to_remove.append(sequences[el][len(sequences[el])-1][0])
    #seq.append(torch.tensor(entry))
    #next.append(target)

seq=torch.nn.utils.rnn.pad_sequence(seq,batch_first=True)
seq_dim = len(seq[0])-4
print(seq_dim)
#seq=torch.nn.utils.rnn.pad_sequence(seq)

print(len(to_remove))

#to_remove_new = []

#for el in to_remove:

#  el = nodeIds[el]
#  to_remove_new[]

print(seq)
print(len(next))



{58: [[585, 16, 2], [901, 12, 1], [2757, 19, 5], [1021, 11, 2], [100, 11, 2], [809, 10, 2], [75, 14, 2]], 465: [[423, 10, 1], [2687, 4, 2], [486, 22, 5], [595, 7, 6], [270, 15, 6]], 293: [[2692, 16, 5], [377, 13, 3], [210, 17, 5], [1184, 20, 1], [210, 20, 1], [1485, 12, 4], [2859, 13, 2], [2533, 13, 4], [2692, 12, 3], [2714, 12, 6], [377, 13, 1], [2533, 20, 2], [2743, 13, 6], [2859, 19, 4], [3745, 19, 1], [2533, 20, 3], [1397, 15, 2], [3840, 0, 4], [377, 16, 1], [395, 11, 3], [3169, 12, 2], [3229, 1, 1], [3887, 12, 2], [2939, 17, 5], [2686, 9, 1], [1167, 12, 2], [2714, 1, 4], [2743, 2, 5], [2533, 20, 1], [3169, 12, 2], [1328, 16, 5], [377, 13, 1], [3169, 17, 5], [2533, 20, 0], [3890, 1, 5], [3169, 12, 0], [516, 20, 4], [1053, 12, 3], [2686, 11, 6], [2766, 11, 6], [1397, 13, 4], [1783, 13, 6], [2718, 10, 3], [2622, 17, 5], [1053, 13, 1], [1053, 13, 0], [2937, 16, 6], [2686, 8, 0], [3840, 0, 2], [3840, 3, 6], [3067, 20, 2], [2686, 13, 5], [3840, 1, 5], [1783, 22, 5], [2718, 10, 6], [210,

In [66]:
df = df[~df['Venue ID'].isin(to_remove)]
venuesdf = venuesdf[~venuesdf['Venue ID'].isin(to_remove)]
venuesdict = venuesdf.to_dict('records')
print(df)
print(venuesdf)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
36506        58       585                  6  40.726443 -73.995746      -240   
96254       465       423                214  40.732876 -73.988193      -240   
16267       293      2692                247  40.689003 -73.929085      -240   
88301       928      1111                 43  40.795185 -73.931266      -240   
149382      333      2140                264  40.707209 -74.014259      -240   
...         ...       ...                ...        ...        ...       ...   
193485      315      3682                 98  40.672956 -73.870842      -300   
221869      742      1136                258  40.705414 -73.949383      -300   
222152      359      3697                298  40.835610 -73.940859      -300   
59983       735      2209                225  40.626383 -74.092626      -240   
134773     1000       208                 70  40.750795 -73.993576      -240   

                              UTC time 

In [67]:
#fast spatial graph construction

import networkx as nx
venuesdict = venuesdf.to_dict('records')
#print(venuesdict)

#for el in venuesdict:
#  print((el,venuesdict[el]))
'''
G = nx.Graph()

distances = []

from tqdm import tqdm

for i in tqdm(range(len(venuesdict) - 1)):
    if not G.has_node(venuesdict[i]['Venue ID']):
        G.add_node(venuesdict[i]['Venue ID'], category=venuesdict[i]['Venue category ID'])

    for j in range(i + 1, len(venuesdict)):
        if not G.has_node(venuesdict[j]['Venue ID']):
            G.add_node(venuesdict[j]['Venue ID'], category=venuesdict[i]['Venue category ID'])

        if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                        venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
            G.add_edge(venuesdict[i]['Venue ID'], venuesdict[j]['Venue ID'])
'''
from tqdm import tqdm
import torch
from torch_geometric.data import Data

edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   #if venuesdict[i]['Venue ID'] not in node_attr:
   #  node_attr.append(venuesdict[i]['Venue ID'])

   for j in range(i + 1, len(venuesdict)):
    #  if venuesdict[j]['Venue ID'] not in node_attr:
    #     node_attr.append(venuesdict[j]['Venue ID'])


      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append([1])

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.LongTensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)



100%|██████████| 3405/3405 [00:37<00:00, 89.93it/s] 


Data(x=[3949, 1], edge_index=[2, 143029], edge_attr=[143029, 1])


In [68]:
import pickle

print(spatG)

#spatpG = G

pickle.dump(spatG,open('spatG.pickle','wb'))

#data = pickle.load(open('spatG.pickle','rb'))

Data(x=[3949, 1], edge_index=[2, 143029], edge_attr=[143029, 1])


In [69]:
#nx.draw(G)
print(spatG)

Data(x=[3949, 1], edge_index=[2, 143029], edge_attr=[143029, 1])


In [70]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}

#seqG = nx.Graph()

for _, row in df.iterrows():

  #if not seqG.has_node(row['Venue ID']):
  #  seqG.add_node(row['Venue ID'], category=row['Venue category ID'])

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    #seqG.add_edge(usrloc[row['User ID']],row['Venue ID'])
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)






Data(x=[3949, 1], edge_index=[2, 4181], edge_attr=[4181])


In [71]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Data(x=[3949, 1], edge_index=[2, 4181], edge_attr=[4181])


In [72]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

#week['workdays'] = []
#week['weekend'] = []


for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  #if 1 <= day <=4:
  #  week['workdays'].append([row['Venue ID'],row['Venue category ID']])
  #else:
  #  week['weekend'].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    #hours[hour] = [[row['Venue ID'],row['Venue category ID']]]
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






      Venue ID  Venue category ID  localHour
0            0                 21       15.0
1            1                 39       23.0
2            2                 47       14.0
3            3                 51       19.0
4            4                200       21.0
...        ...                ...        ...
3419      3944                  5       21.0
3420      3945                139       10.0
3421      3946                 61        8.0
3422      3947                 61        9.0
3423      3948                141       11.0

[3424 rows x 3 columns]
      Venue ID  Venue category ID  localHour  localDayofWeek
0            0                 21       15.0             1.0
1            1                 39       23.0             4.0
2            2                 47       14.0             1.0
3            3                 51       19.0             3.0
4            4                200       21.0             4.0
...        ...                ...        ...             ...
3419    

In [73]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    #if not hourG.has_node(u[0]):
    #  hourG.add_node(u[0], category=u[1])

    for v in hours[el]:

      if u != v:

        #if not hourG.has_node(u[0]):
        #  hourG.add_node(v[0], category=v[1])

        #hourG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)




Data(x=[3949, 1], edge_index=[2, 617032], edge_attr=[0])


In [74]:
pickle.dump(hourG,open('hourG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [75]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    #if not dayG.has_node(u[0]):
    #  dayG.add_node(u[0], category=u[1])

    for v in week[el]:

      if u != v:

        #if not dayG.has_node(u[0]):
        #  dayG.add_node(v[0], category=v[1])

        #dayG.add_edge(u[0],v[0])
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

Data(x=[3949, 1], edge_index=[2, 1713086], edge_attr=[0])


In [76]:
pickle.dump(dayG,open('dayG.pickle','wb'))

In [77]:
!pip install pytorch-lightning
import pytorch_lightning as pl

In [ ]:


import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec

# Lightning Module for Node2Vec model
class Node2VecModel(pl.LightningModule):
    def __init__(self, data, embedding_dim=128, walk_length=20, context_size=10, walks_per_node=10, num_negative_samples=1, p=1.0, q=1.0, lr=0.01):
        super().__init__()
        self.data = data
        self.model = Node2Vec(
            data.edge_index,
            embedding_dim=embedding_dim,
            walk_length=walk_length,
            context_size=context_size,
            walks_per_node=walks_per_node,
            num_negative_samples=num_negative_samples,
            p=p,
            q=q,
            sparse=True
        )
        self.lr = lr

    def forward(self):
        return self.model()

    def training_step(self, batch, batch_idx):
        pos_rw, neg_rw = batch
        loss = self.model.loss(pos_rw.to(self.device), neg_rw.to(self.device))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.SparseAdam(self.parameters(), lr=self.lr)

    def test_step(self, batch, batch_idx):
        z = self.model()
        acc = self.model.test(
            train_z=z[self.data.train_mask],
            train_y=self.data.y[self.data.train_mask],
            test_z=z[self.data.test_mask],
            test_y=self.data.y[self.data.test_mask],
            max_iter=150
        )
        self.log('test_acc', acc)
        return acc




batch_size = 128
num_workers = 4

model = Node2VecModel(data=spatG)

loader = model.model.loader(batch_size=128, shuffle=True, num_workers=num_workers)

trainer = pl.Trainer(
    max_epochs=1,
    callbacks=[
        ModelCheckpoint(monitor='train_loss'),
        EarlyStopping(monitor='train_loss', patience=10)
    ]
)
trainer.fit(model, loader)
#trainer.test(model, loader)

# Plotting points


model.eval()
z = model().detach()
#z = TSNE(n_components=2).fit_transform(z)
print(z.shape)


In [79]:
from sklearn.model_selection import train_test_split

batchsize = 32


#train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2,stratify=torch.stack(next))
train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2)
#train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2,stratify=train_y)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2)

trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)

valset = torch.utils.data.TensorDataset(val_x,val_y)
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)

testset = torch.utils.data.TensorDataset(test_x,test_y)
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)

In [ ]:
#import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class GCN(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)

    def forward(self, x, edge_index):
        #print(x)
        #print("sono una nuova convoluzione")
        #print(edge_index)
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

class Model(nn.Module):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(Model, self).__init__()

        # User Embedding Layer
        self.user_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=64)

        #Last location Embedding Layer
        self.lastLoc_embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=64)

        #Time and day Input linear

        self.timeday_embedding = nn.Linear(2,64)

        # Location Embedding Layers
        self.gcn1 = GCN(spatG.num_features, 64)
        self.gcn2 = GCN(seqG.num_features, 64)
        self.gcn3 = GCN(hourG.num_features, 64)
        self.gcn4 = GCN(dayG.num_features, 64)

        # Sequence Embedding Layer
        self.self_attn = nn.MultiheadAttention(embed_dim=sequence_dim, num_heads=1)

        #Expand sequence representation

        self.expand_sequence = nn.Linear(sequence_dim,64)

        # Multihead Attention Layer
        self.multihead_attn = nn.MultiheadAttention(embed_dim=64, num_heads=8)

        # Classifier Layers
        self.linear1 = nn.Linear(2544, 128)
        self.linear2 = nn.Linear(128, 64)
        self.linear3 = nn.Linear(64, numvenues)

    def forward(self, user, lastloc, lasthour, lastday, sequence):
        #print(lastday)
        curr_batch = len(user)
        print(len(user))
        user_embed = self.user_embedding(user)

        lastloc_embed = self.lastLoc_embedding(lastloc)

        timeday_embed = self.timeday_embedding(torch.stack([lasthour.float(), lastday.float()],dim=1))

        #nodes = spatG.x[lastloc]

        #print(nodes)

        k = 1
        # Location Embeddings
        #nodes,edge_index1,mapping,_ =torch_geometric.utils.k_hop_subgraph(lastloc,k,spatG.edge_index)
        #print(len(nodes))

        #nodes = spatG.x[nodes]

        #print(nodes)

        #print(isinstance(spatG.edge_index, torch.Tensor))
        #print(spatG.edge_index)



        location_embed = self.gcn1(x=spatG.x, edge_index=spatG.edge_index)
        seq_embed = self.gcn2(x=seqG.x, edge_index=seqG.edge_index)
        hour_embed = self.gcn3(x=hourG.x,edge_index=hourG.edge_index)
        day_embed =  self.gcn4(x=dayG.x,edge_index=dayG.edge_index)

        #print(sequence)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence.float(), sequence.float(), sequence.float())

        sequence_embed = self.expand_sequence(sequence_embed)

        print(user_embed.shape)
        print(location_embed.shape)
        print(seq_embed.shape)
        print(hour_embed.shape)
        print(day_embed.shape)
        print(sequence_embed.shape)
        print(lastloc_embed.shape)
        print(timeday_embed.shape)

        #user_embed = user_embed*sequence_embed

        # Concatenate all embeddings
        concat_embed = torch.cat([user_embed,lastloc_embed,location_embed,seq_embed,hour_embed,day_embed,timeday_embed,sequence_embed], dim=0)


        print(concat_embed)
        #print(len(spatG.x))

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        print(attn_output.shape)


        # Classifier
        out = F.relu(self.linear1(attn_output.view(curr_batch,-1)))
        out = F.relu(self.linear2(out))
        out = self.linear3(out)

        return out

# Define Lightning model using PyTorch Lightning
import pytorch_lightning as pl

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim):
        super(LightningModel, self).__init__()
        self.model = Model(input_dim, location_dim, sequence_dim)

    def forward(self, user, timestamp, location, sequence):
        return self.model(user, timestamp, location, sequence)

    def training_step(self, batch, batch_idx):

        x, y = batch
        #print(x)
        y_pred = self.model(x[:,0],x[:,1],x[:,2],x[:,3],x[:,4:])
        loss = F.cross_entropy(torch.sigmoid(y_pred), y)
        #print(loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

# Example of training the model
input_dim = 10000
location_dim = 64
sequence_dim = seq_dim
#print(seq_dim)
model = LightningModel(input_dim, location_dim, sequence_dim)

trainer = pl.Trainer(max_epochs=10)
trainer.fit(model, trainloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type  | Params
--------------------------------
0 | model | Model | 1.9 M 
--------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.727     Total estimated model params size (MB)
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

32
torch.Size([32, 64])
torch.Size([3949, 64])
torch.Size([3949, 64])
torch.Size([3949, 64])
torch.Size([3949, 64])
torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32, 64])
tensor([[ 1.6972e+00,  1.4936e-01, -1.4403e-01,  ...,  5.4302e-01,
         -1.5621e+00,  2.3225e+00],
        [ 8.0199e-01,  1.1397e-01, -6.6293e-01,  ..., -1.0899e-01,
         -4.7082e-01,  3.3341e-01],
        [-6.1356e-01, -1.1938e+00,  2.8864e-01,  ..., -2.0182e+00,
         -4.7878e-01,  1.4319e+00],
        ...,
        [-1.5058e+01, -1.0810e+02, -2.3493e+02,  ...,  1.1213e+02,
         -2.1158e+01,  3.5488e+02],
        [ 3.5644e+01, -1.3848e+02, -4.7021e+02,  ..., -2.3386e+01,
          3.4013e+02,  4.3181e+01],
        [-2.3398e+02, -7.0117e+01, -1.4847e+01,  ...,  2.6285e+02,
         -1.5480e+02,  7.3178e+00]], grad_fn=<CatBackward0>)
